### IMAGE COMPLETION

Image Completion is the task of filling missing parts of a given image with the help of information from the known parts of the image. This is an application that takes an image with a missing part as input and gives a completed image as the result.

We will be using Autoencoder to do this task. We will train our network on Images with missing parts passed with true images so that autoencoder can minimize the ture image and corrupted image. 

### Imports

In [ ]:
import os
import random
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

import tensorflow as tf
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape
from keras.layers import Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import plot_model
from skimage.io import imread
from PIL import Image, ImageDraw

### Data

The data we are using is [Flickr-Faces-HQ Dataset (FFHQ)](https://github.com/NVlabs/ffhq-dataset). It is an unlabelled dataset used for training GANs and other image generation algorithms. The original dataset has images of size 1024 by 1024 but we have only taken 128 by 128 images. 

Mounting your google drive.

In [ ]:
from google.colab import drive
drive.mount('/drive')

Unzipping the data file to load it locally in the colab runtime. You can see your unzipped files by clicking the folder icon on left side of your colab.

In [ ]:
# replace this your google drive path of the zip file of dataset provided with this homework
!unzip -o -q "/drive/MyDrive/CS5317_DeepLearning_SP21/Assign04/ffhq-dataset.zip" -d "/content/data/"

In [ ]:
# data folder path in colab runtime enviroment
DATA_FOLDER = '/content/data/'

# fetching all the filenmaes to read them later in generator
filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.png')))

# total images in directory
NUM_IMAGES = len(filenames)
print("Total number of images : " + str(NUM_IMAGES))

### Data Generator

The dataset is quite large (70000 images) which makes it impossible to load it all at the same time in computer memory. In you assignment 03 you implemented a custom generator function to load the images in batches, here we are going to do the same. Instead of returning the images and its labels, here we will return tuple <i>(corrupted_images_batch, original_images_batch)</i> from the generator where the corrupted images are the same images as the original but a small square is removed from them.

Below you will create a function to remove a portion of image. This is basically the same as drawing a black square on the image. You function will take a numpy image and return the numpy image with black square on it. 


You images will look somehting like this. The square drawn here is 28x28 (you can be confortable with the dimensions) and it is drawn at a random location with-in the image.

![picture](https://drive.google.com/uc?export=view&id=1qIlWIj1K_qjxoGTUJYeVbiLztnyMOVP_)


<i>HINT: You can use ImageDraw function of PIL</i>

In [ ]:
def draw_square_on_image(image):
    
    output_img = None
    
    ######################## WRITE YOUR CODE BELOW ########################

    ########################### END OF YOUR CODE ##########################

    return np.array(output_img)

If you are not fimiliar with generators in python you can a look at it [here](https://realpython.com/introduction-to-python-generators/).

As mentioned above, below generator will return a tuple of <i>(corrupted_images_batch, original_images_batch)</i>.

In [ ]:
def custom_image_generator(files, data_instances, batch_size = 64):

    ######################## WRITE YOUR CODE BELOW ########################

    # to keep track that you don't have invalid index for number of files
    iter = 0

    while True:

        # check if you have a invalid index for files, if yes then reset it


        # Select files (paths/indices) for the batch

         
        # Read in each input and perform preprocessing (to batch of images)


        # Return a tuple of (corrupted_image_batch, true_image_batch) to feed the network
        corrupted_images_batch = np.array(batch_input)
        original_images_batch = np.array(proc_batch)

        # move to the next batch
        iter = iter + 1

    ########################### END OF YOUR CODE ##########################
    
        yield (corrupted_images_batch, original_images_batch)

Utility function to display grid of images.

In [ ]:
def display_image_grid(images, num_rows, num_cols, title_text):

    fig = plt.figure(figsize=(num_cols*3., num_rows*3.), )
    grid = ImageGrid(fig, 111, nrows_ncols=(num_rows, num_cols), axes_pad=0.15)

    for ax, im in zip(grid, images):
        ax.imshow(im)
        ax.axis("off")
    
    plt.suptitle(title_text, fontsize=20)
    plt.show()

In [ ]:
# create generator object
test_generator = custom_image_generator(filenames, 7000)

Displaying sample images from batch generator.

In [ ]:
# get first batch of images
(corrupted_images_batch, orig_images_batch) = next(test_generator)

# only displaying 10 images from both batch
display_image_grid(orig_images_batch[:10], 2, 5, "Original Images")
display_image_grid(corrupted_images_batch[:10], 2, 5, "Corrupted Images")

In [ ]:
INPUT_DIM = (128,128,3) # Image dimension
Z_DIM = 300             # Dimension of the latent vector (z)

## Building the Model

#### Encoder

Below you will create the model for the encoder. The architecture of the Encoder consists of a stack of convolutional layers followed by a dense (fully connected) layer which outputs a vector of size <b>Z_DIM</b>. The whole image of size 128x128x3 is decoed into this latent space vector of size <b>Z_DIM</b>.

for i=1 to num_conv:
  - add Covn Layer (filter_size = 32, stride = 2, padding = 'same')
  - add LeakeyReLU

end
- add Dense() (with no activation function)

NOTE: You can also experiment with the number of feature maps, kernel size and strides for each of the conv layer.

You can refer to this [link](https://blog.keras.io/building-autoencoders-in-keras.html) to see how to create autoencoer model in keras.

In [ ]:
ae_encoder = None
ae_encoder_output = None
ae_encoder_input = None


######################## WRITE YOUR CODE BELOW ########################


########################### END OF YOUR CODE ##########################

ae_encoder.summary()
# tf.keras.utils.plot_model(ae_encoder, show_shapes=True)

#### Decoder

Just like the encoder you will create the model for the decoder. This model will be the exact mirror of encoder model, but that is not mandatory.

Since the function of the Decoder to reconstruct the image from the latent vector. Therefore, it is necessary to define the decoder so as to increase the size of the activations gradually through the network. This can be achieved through the  [Conv2DTransponse](https://keras.io/layers/convolutional/#conv2dtranspose) layer. This layer produces an output tensor double the size of the input tensor in both height and width. The input to the encoder is the vector of size <b>Z_DIM</b> and output will be a image of size <b>INPUT_DIM</b>. Your final decoder will look something like this: 

<center>

![picture](https://drive.google.com/uc?export=view&id=1QGaPm7byp7YOZqrZx9ARX9hPBVHB9rq5)

</center>

Again, you can experiment with the number of layers, feature size, kernel size and stride of conv layers.

<i>NOTE: Unlike the encoder, there will the activaiton function for decoder, as it will be outputing the image. And we want our pixel values between zero and one. </i>


In [ ]:
ae_decoder = None
ae_decoder_output = None

######################## WRITE YOUR CODE BELOW ########################


########################### END OF YOUR CODE ##########################

ae_decoder.summary()
# tf.keras.utils.plot_model(ae_decoder, show_shapes=True)

#### Attaching the Decoder to the Encoder

Finally, here we connect the encoder to the docoder.

In [ ]:
autoencoder_model = None

######################## WRITE YOUR CODE BELOW ########################

# The input of the autoencoder will be the same as of encoder


# The output of the autoencoder will be the output of decoder, when passed encoder input


# Input to the combined model will be the input to the encoder.
# Output of the combined model will be the output of the decoder.

########################### END OF YOUR CODE ##########################

autoencoder_model.summary()

## Training the AE

Finally you will compile your autoencoer here. 

Here are few hyperparamters to consider here:
- Learning rate [0.1, 0.00001]
- Training epochs [5, 50]
- batch_size [64, 512]
- Latent vector size [20, 5000]
- Error function 
- Optimizer

In [ ]:
LEARNING_RATE = None    # learning rate
N_EPOCHS = None         # epochs
BATCH_SIZE = None       # batch of images returned by image generator

######################## WRITE YOUR CODE BELOW ########################

# create custom_data_generator here


# compile your model here

########################### END OF YOUR CODE ##########################

Now simply call the <i>fit</i> function of the model with the appropriate paramters.

<i> HINT: Pass step_per_epoch to be equal to total images divided by batch_size. As you want to see all your data in a single epoch. </i>

In [ ]:
######################## WRITE YOUR CODE BELOW ########################


########################### END OF YOUR CODE ##########################

## Reconstruction

Now we will get a batch of images from data generator object and try to reproduce it by passing through AE.

The first image grid shows the original images and the second grid shows the reconstructed images after passing it through the AE.

In [ ]:
test_gen = custom_image_generator(filenames, NUM_IMAGES)

test_batch = next(test_gen)[0]
test_images = test_batch[:10]

reconst_images = autoencoder_model.predict(test_images)

display_image_grid(test_images, 2, 5, "Incomplete Images")
display_image_grid(reconst_images, 2, 5, "Reconstructed Images with Autoencoder")

## REPORT

Report your results for different values of <b>Z_DIM</b>, <b>learning rate</b>, <b>optimizers</b>, <b> encoder and decoder model</b> and tell us for which configuration you acheived the best results (The best run model should be the last run model in this notebook, showing the results in the cell above).

Your answer: